In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import datetime
import re

### cargar archivo 1

In [5]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padon_1)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/UOETSYLRA JUNIO.xlsx' mode='r' encoding='cp1252'>
      Codigo  CUIT_Empleador  CUIL_Titular  Parentesco  CUIL_Familiar  \
0     127406     20240830214   20240830214           0    20240830214   
1     127406     20119590958   20119590958           0    20119590958   
2     127406     27425694192   27425694192           0    27425694192   
3     127406     27425694192   27425694192           3    20549159851   
4     127406     27270992132   27270992132           0    27270992132   
...      ...             ...           ...         ...            ...   
1682  127406     20190072577   20190072577           0    20190072577   
1683  127406     27321459590   27321459590           0    27321459590   
1684  127406     27174774221   27174774221           0    27174774221   
1685  127406     27174774221   27174774221           1    20149595150   
1686  127406     27316755637   27316755637           0    27316755637   

     Tipo_Doc   Nro_Doc

In [4]:
print(padon_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CUIL TITULAR*        1119 non-null   object 
 1   CUIL*                567 non-null    float64
 2   APELLIDO*            1726 non-null   object 
 3   NOMBRES*             1726 non-null   object 
 4   DOMICILIO (calle)*   1698 non-null   object 
 5   Domicilio (altura)*  1667 non-null   object 
 6   LOCALIDAD*           1705 non-null   object 
 7   TELEFONO             1215 non-null   object 
 8   N° DE FORMULARIO     1614 non-null   object 
 9   FECHA DE ALTA*       1664 non-null   object 
 10  FECHA DE NAC.*       1549 non-null   object 
 11  NACIONALIDAD*        1725 non-null   float64
 12  Sexo*                1723 non-null   object 
 13  Estado Civil*        1667 non-null   float64
 14  Parentesco*          1711 non-null   float64
 15  SEDE                 0 non-null      f

### cargar archivo 2

In [20]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron_2 = pd.read_excel(ruta_archivo, engine='xlrd')
            padron_2 = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron_2)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/padron_completo_13-06-23.csv' mode='r' encoding='cp1252'>
       Nro. OS      Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7            Desregulado     SI             26407599   
1            7            Desregulado     SI             26407599   
2            6               Afiliado     SI             30694069   
3           10            Desregulado     SI             31403090   
4            1  Monotributista social     NO             36821087   
...        ...                    ...    ...                  ...   
15296        4            Desregulado     SI             21601831   
15297        4            Desregulado     SI             21601831   
15298        4            Desregulado     SI             21601831   
15299        4            Desregulado     SI             21601831   
15300        1               Afiliado     SI             36602782   

         Nro. Afiliado Parentesco con el TITULAR Apellido Afili

### Normalizar DNI archivo convenio

In [21]:

#armar el camp DNI juntando las los campos cuil titular y cuil familiar
padon_1['dni']=padon_1['CUIL TITULAR*'].fillna(padon_1['CUIL*'])
#limpiar el nuevo campo dni dejando solo los numeros 
padon_1['dni']=padon_1['dni'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
#dejar solo los numreros eliminar caracteres 
padon_1['dni2']=padon_1['dni'].apply(lambda x: x[2:10] if len(x)>= 8 else '0')
#extraer el dni del cuil y dejar vacios los q son menores  a 8
padon_1['dni2']=padon_1['dni2'].astype(int)
#pasar el campo a int 
padon_1['FECHA DE NAC.*']=padon_1['FECHA DE NAC.*'].astype(str)
#convertir fecha. 
print(padon_1['FECHA DE NAC.*'])
padon_1=funciones.transformar_fecha(padon_1, 'FECHA DE NAC.*')
#print(padon_1[['FECHA DE ALTA*','FECHA DE NAC.*','dni2']].head(30))q   


0       1999-02-12 00:00:00
1       1974-11-03 00:00:00
2       1985-09-19 00:00:00
3       1991-03-20 00:00:00
4       2001-02-03 00:00:00
               ...         
1721                    nan
1722                    nan
1723                    nan
1724                    nan
1725                    nan
Name: FECHA DE NAC.*, Length: 1726, dtype: object


c:\Users\zickd\OneDrive\Documentos\repositoriosGit\osoetsy_csv\funciones.py:32: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df[columna]=pd.to_datetime(df[columna], dayfirst=False, errors='coerce')


### Errores

In [ ]:
condiciones = [
    lambda row: row['dni'] is None or len(str(row['dni'])) < 8,
    lambda row: row['FECHA DE NAC.*'] is None or (pd.to_datetime('today') - pd.to_datetime(row['FECHA DE NAC.*'])).days // 365.25 > 80,
    lambda row: row['CUIL*'] >999999 and (pd.to_datetime('today') - pd.to_datetime(row['FECHA DE NAC.*'])).days / 365.25 > 25,
    lambda row: row['LOCALIDAD*'] is None
]
mensajes = [
    'Error en DNI',
    'Error en la edad',
    'familiar mayor de edad',
    'localidad vacía'
]

def generar_columnas_error(row):
    errores = {}
    for i in range(len(condiciones)):
        if condiciones[i](row):
            columna_error = f'error{i+1}'
            errores[columna_error] = mensajes[i]
    return pd.Series(errores)

df_errores = padon_1.apply(generar_columnas_error, axis=1)


df_nuevo = pd.concat([padon_1, df_errores], axis=1)

df_sin_error = df_nuevo[df_nuevo['error1'].notna() | df_nuevo['error2'].notna() | df_nuevo['error3'].notna()]

df_sin_error.to_excel(r'C:\Users\zickd\Downloads\test_errores.xlsx', engine='openpyxl', index=False)

df_sin_errores = df_nuevo[df_nuevo['error1'].isnull() & df_nuevo['error2'].isnull() & df_nuevo['error3'].isnull()]

#df_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)

df_sin_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)


### Merge archivos x DNI

In [44]:
cruce_tis=pd.merge(padon_1, padron_2, left_on='dni2', right_on='Nro. Documento', how='left', indicator=True)
#cruce_tis.to_excel(r'C:\Users\zickd\Downloads\merge.xlsx', engine='openpyxl', index=False)

cruce_tis_filtro=cruce_tis[cruce_tis['_merge']=='left_only']

#print(cruce_tis_filtro.head(5))

### exportar el cruce

In [46]:
save_path = filedialog.askdirectory()
#uso iloc por si las columnas tienen el mismo nombre, necesito la primeras 20 columnas
cruce_tis_filtro.iloc[:, :18].to_excel(save_path + '\cruce_tis.xlsx', engine='openpyxl' ,index = False )

